<a href="https://colab.research.google.com/github/A-R-I-N-D-A-M/intermediate/blob/master/Information_retrieval_using_w2v_and_ELMo_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
training_queries=pd.read_csv('/content/drive/MyDrive/Dataset/dataset/queries.csv')
training_queries.head()

,qid,query
0,1,what similarity laws must be obeyed when const...
1,2,what are the structural and aeroelastic proble...
2,3,what problems of heat conduction in composite ...
3,8,what methods -dash exact or approximate -dash ...
4,10,are real-gas transport properties for air avai...


In [ ]:
testing_queries=pd.read_csv('/content/drive/MyDrive/Dataset/dataset/queries_val.csv')
testing_queries.head()

,qid,query
0,189,is there a design method for calculating therm...
1,190,will an analysis of panel flutter based on arb...
2,191,"what is the criterion for true panel flutter, ..."
3,194,how can the analytical solution of the bucklin...
4,196,the problem of similarity for representative i...


In [ ]:
train_top5=pd.read_csv('/content/drive/MyDrive/Dataset/dataset/qrel.csv')
train_top5.head()

,qid,docid,rank
0,1,184,1
1,1,29,2
2,1,31,3
3,1,57,4
4,1,378,5


In [ ]:
test_top5=pd.read_csv('/content/drive/MyDrive/Dataset/dataset/qrel_val.csv')
test_top5.head()

,qid,docid,rank
0,189,395,1
1,189,866,2
2,189,869,3
3,189,865,4
4,189,868,5


In [ ]:
rel=list(range(1,4))
train_top5['rel']=train_top5['rank'].apply(lambda x: 1 if x in rel else 0)
test_top5['rel']=test_top5['rank'].apply(lambda x: 1 if x in rel else 0)

In [ ]:
train_top5['rel']=train_top5['rel'].astype(int)
test_top5['rel']=test_top5['rel'].astype(int)

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Dataset/dataset/documents.csv')
df.head()

,docid,author,bibliography,body,title
0,2,ting-yili,"department of aeronautical engineering, rensse...",simple shear flow past a flat plate in an inco...,simple shear flow past a flat plate in an inco...
1,3,m. b. glauert,"department of mathematics, university of manch...",the boundary layer in simple shear flow past a...,the boundary layer in simple shear flow past a...
2,5,"wasserman,b.","j. ae. scs. 24, 1957, 924.",one-dimensional transient heat conduction into...,one-dimensional transient heat conduction into...
3,6,"campbell,w.f.","j. ae. scs. 25, 1958, 340.",one-dimensional transient heat flow in a multi...,one-dimensional transient heat flow in a multi...
4,12,"bisplinghoff,r.l.","j. ae. scs. 23, 1956, 289.",some structural and aerelastic considerations ...,some structural and aerelastic considerations ...


In [ ]:
df.drop(['author','bibliography','title'],axis=1,inplace=True)

In [ ]:
df.body[0]

"simple shear flow past a flat plate in an incompressible fluid of small viscosity . in the study of high-speed viscous flow past a two-dimensional body it is usually necessary to consider a curved shock wave emitting from the nose or leading edge of the body .  consequently, there exists an inviscid rotational flow region between the shock wave and the boundary layer .  such a situation arises, for instance, in the study of the hypersonic viscous flow past a flat plate .  the situation is somewhat different from prandtl's classical boundary-layer problem . in prandtl's original problem the inviscid free stream outside the boundary layer is irrotational while in a hypersonic boundary-layer problem the inviscid free stream must be considered as rotational .  the possible effects of vorticity have been recently discussed by ferri and libby .  in the present paper, the simple shear flow past a flat plate in a fluid of small viscosity is investigated .  it can be shown that this problem ca

In [ ]:
training_corpus=df[df['docid'].isin(train_top5['docid'].unique())].reset_index(drop=True)

In [ ]:
training_corpus

,docid,body
0,2,simple shear flow past a flat plate in an inco...
1,3,the boundary layer in simple shear flow past a...
2,5,one-dimensional transient heat conduction into...
3,6,one-dimensional transient heat flow in a multi...
4,12,some structural and aerelastic considerations ...
...,...,...
319,1326,interaction of secondary injectants and rocket...
320,1355,boundary layer displacement effects in air at ...
321,1364,an experimental investigation of the interacti...
322,1378,blunt-cone pressure distributions at hypersoni...


In [ ]:
testing_corpus=df[df['docid'].isin(test_top5['docid'].unique())].reset_index(drop=True)

In [ ]:
testing_corpus

,docid,body
0,15,on two-dimensional panel flutter . theory an...
1,24,theory of stagnation point heat transfer in di...
2,40,experiments on boundary layer transition at su...
3,51,theory of aircraft structural models subjected...
4,58,pressure measurements on sharp and blunt 5 and...
...,...,...
82,1393,heat transfer near the forward stagnation poin...
83,1394,stagnation point heat transfer measurements in...
84,1395,low density stagnation point heat transfer mea...
85,1397,critical shear stress of an infinitely long si...


In [ ]:
import re
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import spacy
nlp=spacy.load('en_core_web_sm')

In [ ]:
  def clean_text(text):
    text = text.lower()
    
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    text = pattern.sub('', text)
    text = " ".join(filter(lambda x:x[0]!='@', text.split()))
    return text
def CleanTokenize(text):
    head_lines = list()
    lines = text.values.tolist()

    for line in lines:
        line = clean_text(line)
        # tokenize the text
        tokens = word_tokenize(line)
        # remove puntuations
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in tokens]
        # remove non alphabetic characters
        words = [word for word in stripped if word.isalpha()]
        stop_words = set(stopwords.words("english"))
        # remove stop words
        words = [w for w in words if not w in stop_words]
        words=' '.join(words)
        head_lines.append(words)
    return head_lines

In [ ]:
body=CleanTokenize(training_corpus.body)
training_corpus['cleaned']=body

In [ ]:
body=CleanTokenize(testing_corpus.body)
testing_corpus['cleaned']=body

In [ ]:
queries=CleanTokenize(training_queries['query'])
training_queries['cleaned']=queries

In [ ]:
queries=CleanTokenize(testing_queries['query'])
testing_queries['cleaned']=queries

In [ ]:
combined_training=pd.concat([training_corpus.rename(columns={'cleaned':'text'})['text'],training_queries.rename(columns={'cleaned':'text'})['text']]).sample(frac=1).reset_index(drop=True)

In [ ]:
combined_training

0      approximate solution hypersonic laminar bounda...
1      boundary layer flat plate shear flow induce pr...
2      dissociation scaling nonequilibrium blunt nose...
3      hypersonic strong viscous interaction flat pla...
4      experiments tapered sweptback wing warren plan...
                             ...                        
404    exist good basic treatment dynamics reentry co...
405    simplified method elastic stability analysis t...
406    tunnel interference effects problems solid blo...
407    analytical study tumbling motions vehicles ent...
408    analytical experimental investigation stress d...
Name: text, Length: 409, dtype: object

# Using W2V model

In [ ]:
from gensim.models import Word2Vec
train_data=[]
for i in combined_training:
  train_data.append(i.split())
w2v_model=Word2Vec(train_data,size=100,window=5,sg=1)

In [ ]:
len(train_data)

409

In [ ]:
print('Vocabulary size is :',len(w2v_model.wv.vocab))

Vocabulary size is : 1193


In [ ]:
def get_embedding(tokens):
  embeddings=[]
  if len(tokens)<1:
    return np.zeros(100)
  else:
    for token in tokens:
      if token in w2v_model.wv.vocab:
        embeddings.append(w2v_model.wv.word_vec(token))
      else:
        embeddings.append(np.random.rand(100))
    return np.mean(embeddings,axis=0)

In [ ]:
testing_corpus['vector']=testing_corpus['cleaned'].apply(lambda x: get_embedding(x.split()))
testing_queries['vector']=testing_queries['cleaned'].apply(lambda x: get_embedding(x.split()))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Function for calculating average precision for a query
def average_precision(qid,qvector):
  
  # Getting the ground truth and document vectors
  qresult=test_top5.loc[test_top5['qid']==qid,['docid','rel']]
  qcorpus=testing_corpus.loc[testing_corpus['docid'].isin(qresult['docid']),['docid','vector']]
  qresult=pd.merge(qresult,qcorpus,on='docid')
  
  # Ranking documents for the query
  qresult['similarity']=qresult['vector'].apply(lambda x: cosine_similarity(np.array(qvector).reshape(1, -1),np.array(x).reshape(1, -1)).item())
  qresult.sort_values(by='similarity',ascending=False,inplace=True)

  # Taking Top 10 documents for the evaluation
  ranking=qresult.head(10)['rel'].values
  
  # Calculating precision
  precision=[]
  for i in range(1,6):
    if ranking[i-1]:
      precision.append(np.sum(ranking[:i])/i)
  
  # If no relevant document in list then return 0
  if precision==[]:
    return 0

  return np.mean(precision)

In [ ]:
# Calculating average precision for all queries in the test set
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)

# Finding Mean Average Precision
print('Mean Average Precision :',testing_queries['AP'].mean())

Mean Average Precision : 0.7434343434343433


In [ ]:
def ranking_ir(text):
  query=clean_text(text)
  vector=get_embedding(query)
  document=testing_corpus[['docid','body']].copy()
  document['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1,-1),np.array(x).reshape(1,-1)).item())
  document.sort_values(by='similarity',ascending=False,inplace=True)
  return document.reset_index(drop=True)

In [ ]:
text="what are the nonequilibrium chemical constituents in the viscous shock layer ahead of a blunt re-entry vehicle ."

In [ ]:
ranking_ir(text)

,docid,body,similarity
0,1174,general theory of buckling . various aspects o...,0.986623
1,865,a study of the thermal fatigue behaviour of me...,0.985936
2,875,models for aeroelastic investigation . this ...,0.970383
3,1296,non-equilibrium expansions of air with coupled...,0.968321
4,869,the calculation of transient temperature in tu...,0.964525
...,...,...,...
82,885,buckling of thin cylindrical shells under hoop...,0.534550
83,887,buckling due to thermal stress of cylindrical ...,0.533257
84,1393,heat transfer near the forward stagnation poin...,0.484149
85,1171,the stability under axial compression and late...,0.415034


# Using ELMo embeddings

In [ ]:
!pip install 'tensorflow == 1.15.0'

     |██████████████████████          | 284.8MB 1.5MB/s eta 0:01:23ERROR: Exception:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 382, in run
    resolver.resolve(requirement_set)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/legacy_resolve.py", line 201, in resolve
    self._resolve_one(requirement_set, req)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/legacy_resolve.py", line 365, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/legacy_resolve.py", line 313, in _get_abstract_dist_for
    req, self.session, self.finder, self.require_hashes
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/operations/prepare.py", line 194, in prepare_linked_

In [ ]:
import tensorflow_hub as hub
import tensorflow as tf

In [ ]:
tf.__version__

'2.4.1'

In [ ]:
elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)
def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [ ]:
list_train = [testing_corpus[i:i+50] for i in range(0,testing_corpus.shape[0],50)]
elmo_train = [elmo_vectors(x['cleaned']) for x in list_train]
elmo_train_new = np.concatenate(elmo_train, axis = 0)

In [ ]:
elmo_test = elmo_vectors(testing_queries['cleaned'])

In [ ]:
testing_queries['vector']=elmo_test.tolist()
testing_corpus['vector']=elmo_train_new.tolist()

In [ ]:
testing_queries['AP']=testing_queries.apply(lambda x: average_precision(x['qid'],x['vector']),axis=1)
print('Mean Average Precision :',testing_queries['AP'].mean())

In [ ]:
def embed_elmo2(module):
    with tf.Graph().as_default():
        sentences = tf.placeholder(tf.string)
        embed = hub.Module(module)
        embeddings = embed(sentences)
        session = tf.train.MonitoredSession()
    return lambda x: session.run(embeddings, {sentences: x})
embed_fn = embed_elmo2("https://tfhub.dev/google/elmo/2")

In [ ]:
def ranking_ir_elmo(text):
  query=clean_text(text)
  vector=embed_fn([query])
  document=testing_corpus[['docid','body']].copy()
  document['similarity']=testing_corpus['vector'].apply(lambda x: cosine_similarity(np.array(vector).reshape(1,-1),np.array(x).reshape(1,-1)).item())
  document.sort_values(by='similarity',ascending=False,inplace=True)
  return document.reset_index(drop=True)

In [ ]:
text="what are the nonequilibrium chemical constituents in the viscous shock layer ahead of a blunt re-entry vehicle ."
ranking_ir_elmo(text)